# Heat Index Applications 

This notebook walks through the [NOAA Heat Index](https://www.weather.gov/ama/heatindex) using climate data projections in the Analytics Engine.  

The Heat Index is computed using the NOAA Heat Index equation ([click here to visit NOAA Heat Index page](https://www.wpc.ncep.noaa.gov/html/heatindex_equation.shtml)). The source code for this calculation can be found in the climakitae tools module ([click here to see source code](https://github.com/cal-adapt/climakitae/blob/main/climakitae/tools/indices.py#L48)). The Heat Index is calculated on-the-fly whenever `get_data()` is run as demonstrated in this notebook.   

To execute a given 'cell' of this notebook, place the cursor in the cell and press the 'play' icon, or simply press shift+enter together. Some cells will take longer to run, and you will see a [$\ast$] to the left of the cell while AE is still working.

**Intended Application**: As a user, I want to **<span style="color:#FF0000">understand trends in Heat Index across my region</span>** by:
1. Calculating the number of hours per day throughout the year of high Heat Index values
2. Understanding the trend in nighttime temperatures that are above an 80°F Heat Index
3. Determining the historical and projected number of days with a high Heat Index per month

**Runtime**: With the default settings, this notebook takes approximately **8 minutes** to run from start to finish. Modifications to selections may increase the runtime. 

## Step 0: Set-up

First, we'll import the python library [climakitae](https://github.com/cal-adapt/climakitae), our AE toolkit for climate data analysis, along with this specific functions from that library that we'll use in this notebook, as well as any other necessary python libraries to aid in analysis.

In [1]:
import climakitae as ck
import climakitaegui as ckg
from climakitae.util.utils import (
    read_csv_file,
    get_closest_gridcell,
    compute_multimodel_stats,
    trendline,
    summary_table,
    convert_to_local_time,
)

from climakitae.core.data_interface import get_data
from climakitae.core.paths import STATIONS_CSV_PATH

## Step 1: Select data

### 1a) Grab location of interest by latitude and longitude
First we'll grab specific locations of interest, by using the latitude and longitude of a weather station, and provide code to input a custom lat-lon location. Furthermore, we will **not** be retrieving the actual station data that is bias-corrected to that station for this example. At present bias-corrected station data on the Analytics Engine only provides air temperature as a variable, and for Heat Index we must also have either dew point temperature (coming soon!) or relative humidity. So for the time being, we will retrieve **non-bias corrected** data at the location of interest.

Note: For demonstration purposes we are selecting only ten years of data, plus an extra year that will be dropped after converting to local time. We recomend using a longer period (at least 30 years) for conducting a scientific analysis.

In [ ]:
# select historical data
'''heatidx_hist_hour = get_data(
    variable="NOAA Heat Index",
    resolution="9 km",
    timescale="hourly",
    data_type="Gridded",
    area_subset="CA Electric Load Serving Entities (IOU & POU)",
    cached_area=["Pacific Gas & Electric Company"],
    scenario=["Historical Climate"],
    time_slice=(1981, 1991),  # Short period for faster demonstration
)'''
cd = ck.ClimateData()
heatidx_hist_hour_new_cor = (cd
    .catalog("cadcat")       # Catalog name, using `cadcat` from here on out for the notebook
    .activity_id("WRF")      # Downscaling method (WRF or LOCA)
    .experiment_id("historical")
    .institution_id("UCLA")  # Institution name (WRF -> UCLA, LOCA2 -> UCSD)
    .table_id("1hr")         # Data frequency (`1hr` only available for WRF, `day` and `mon` available for both WRF and LOCA2
    .grid_label("d01")       # Grid resolution (d01=45km, d02=9km, d03=3km)
    .variable("heat_index")          # Variable name (t2 = Air Temperature at 2m)
    .processes({
        "time_slice": (1981, 1981),
        "clip": "KFAT"  # Get closest grid cell to station
    })
).get()

2026-02-13 00:36:10 - climakitae.new_core.user_interface - INFO - Initializing ClimateData interface
2026-02-13 00:36:10 - climakitae.new_core.dataset_factory - INFO - DatasetFactory initialized with 3 validators and 11 processors
2026-02-13 00:36:10 - climakitae.new_core.user_interface - INFO - ClimateData initialization successful
2026-02-13 00:36:10 - climakitae.new_core.user_interface - INFO - ✅ Ready to query!
2026-02-13 00:36:10 - climakitae.new_core.user_interface - INFO - Catalog set to: cadcat
2026-02-13 00:36:10 - climakitae.new_core.user_interface - INFO - Activity ID set to: WRF
2026-02-13 00:36:10 - climakitae.new_core.user_interface - INFO - Experiment ID(s) set to: ['historical']
2026-02-13 00:36:10 - climakitae.new_core.user_interface - INFO - Institution ID set to: UCLA
2026-02-13 00:36:10 - climakitae.new_core.user_interface - INFO - Table ID set to: 1hr
2026-02-13 00:36:10 - climakitae.new_core.user_interface - INFO - Grid label set to: d01
2026-02-13 00:36:10 - clim

In [50]:
heatidx_hist_hour

<xarray.DataArray np.str_('NOAA Heat Index') (scenario: 1, simulation: 8,
                                              time: 78912)> Size: 3MB
array([[[35.141632, 34.51515 , 39.817543, ..., 43.263214, 43.75809 ,
         42.68827 ],
        [35.053658, 34.544216, 34.00293 , ..., 35.162193, 34.335144,
         33.342033],
        [45.38539 , 44.788155, 43.29524 , ..., 35.882385, 34.67431 ,
         34.85522 ],
        ...,
        [39.25213 , 38.915344, 38.24809 , ..., 36.14499 , 35.16478 ,
         34.379738],
        [42.439198, 41.081093, 39.833622, ..., 42.213875, 43.65217 ,
         42.568447],
        [36.58724 , 36.47207 , 34.26487 , ..., 43.369076, 43.36436 ,
         43.291977]]], shape=(1, 8, 78912), dtype=float32)
Coordinates:
  * scenario           (scenario) <U18 72B 'Historical Climate'
  * simulation         (simulation) <U26 832B 'WRF_EC-Earth3_r1i1p1f1' ... 'W...
  * time               (time) datetime64[ns] 631kB 1981-01-01 ... 1990-01-01T...
    x                  float64 8B -4.053e+06
    y                  float64 8B 1.169e+06
    lakemask           float32 4B 0.0
    landmask           float32 4B 1.0
    lat                float32 4B 36.78
    lon                float32 4B -119.7
    Lambert_Conformal  int64 8B 0
Attributes:
    variable_id:           noaa_heat_index_derived
    extended_description:  Operational measure of what the the temperature "f...
    units:                 degF
    data_type:             Gridded
    resolution:            9 km
    frequency:             hourly
    location_subset:       ['Pacific Gas & Electric Company']
    approach:              Time
    downscaling_method:    Dynamical
    institution:           Multiple
    grid_mapping:          Lambert_Conformal
    timezone:              America/Los_Angeles

In [ ]:
heatidx_hist_hour_new_cor

We will first look at the Fresno Airport weather station.

In [3]:
# TODO: Use new core to list stations and move to top
wx_stns = read_csv_file(STATIONS_CSV_PATH, index_col=[0])
wx_stns.head(5)

,state,station,city,ID,LAT_Y,LON_X,station id,elevation
0,CA,Bakersfield Meadows Field (KBFL),Bakersfield,KBFL,35.43424,-119.05524,72384023155,149.3
1,CA,Blythe Asos (KBLH),Blythe,KBLH,33.61876,-114.71451,74718823158,120.4
2,CA,Burbank-Glendale-Pasadena Airport (KBUR),Burbank,KBUR,34.19966,-118.36543,72288023152,222.7
3,CA,Needles Airport (KEED),Needles,KEED,34.76783,-114.61842,72380523179,270.6
4,CA,Fresno Yosemite International Airport (KFAT),Fresno,KFAT,36.77999,-119.72016,72389093193,101.9


In [4]:
station_name = "Fresno Yosemite International Airport (KFAT)"
one_stn = wx_stns.loc[wx_stns["station"] == station_name]

stn_lat = one_stn.LAT_Y.values[0]
stn_lon = one_stn.LON_X.values[0]
print(stn_lat, stn_lon)

36.77999 -119.72016


If you would like to provide your own latitude and longitude coordinates, you can also customize the cell below and pass your own values. However, if your location is outside of the default cached area (for example, we're looking at the PG&E service territory), you'll also need to reset the `cached_area` to one that is more appropriate when you call `get_data()` above.

In [5]:
# stn_lat = YOUR_LAT_HERE
# stn_lon = YOUR_LON_HERE

Now we'll use the latitude and longitude values to retrieve the historical data at that gridcell. 

In [6]:
heatidx_hist_hour = get_closest_gridcell(heatidx_hist_hour, stn_lat, stn_lon)

Closest gridcell to lat: 36.77999, lon: -119.72016 is at lat: 36.783531188964844, lon: -119.73358154296875


Next, we convert the data's time axis to use the local time of the selected station. The dynamically downscaled WRF data in the Cal-Adapt: Analytics Engine is in UTC time. Using the local time is particularly important for determining the timing of the daily maximum and minimum temperatures. For a station located in Pacific Time (US), UTC time places the daily minimum "in" the day prior because UTC is 8 hours ahead of Pacific! The handy `convert_to_local_time` function corrects for this, and ensures that the resulting high and low temperatures are within the same daily timestamp.  

After running convert_to_local_time we drop the final year from the Heat Index timeseries. This is because the final 8 hours will be missing from the final day due to the offset between UTC and Pacific time. By dropping the last year we will have ten years of complete data.

In [7]:
heatidx_hist_hour = convert_to_local_time(heatidx_hist_hour)
heatidx_hist_hour = heatidx_hist_hour.sel({"time": slice("1981-01-01", "1990-01-01")}) # Update this with your own selections to match the data you retrieved!

Data converted to America/Los_Angeles timezone.


Finally we load the data into memory. This step is quick for the small sample dataset used in the demo, but may take several minutes for larger datasets.

In [8]:
heatidx_hist_hour = ck.load(heatidx_hist_hour)

Processing data to read 2.41 MB of data into memory... Complete!


### 1b) Retrieve the projected Heat Index data
Next we will repeat the same data retrieval as we did for the historical data so we can compare. Like the historical data, retrieving the hourly projections data will also take a few minutes - hang tight!

In [9]:
# select future data
heatidx_proj_hour = get_data(
    variable="NOAA Heat Index",
    resolution="9 km",
    timescale="hourly",
    data_type="Gridded",
    approach="Time",
    scenario=["SSP 3-7.0"],
    area_subset="CA Electric Load Serving Entities (IOU & POU)",
    cached_area=["Pacific Gas & Electric Company"],
    time_slice=(2040, 2050),  # Short period for quick demonstration
)

/home/jovyan/.local/lib/python3.12/site-packages/climakitae/core/data_load.py:591: FutureWarning: In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): 'time' ('time',) The recommendation is to set join explicitly for this case.
  one_scenario = xr.concat(
/home/jovyan/.local/lib/python3.12/site-packages/climakitae/core/data_load.py:591: FutureWarning: In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): 'time' ('time',) The recommendation is to set join explicitly for this case.
  one_scenario = xr.concat(
/home/jovyan/.local/lib/python3.12/site-packages/c

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!! Returned data array is huge. Operations could take 10x to infinity longer than 1GB of data !!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



Following the procedure used for the historical data, we will select the data at the grid cell closest to the station.

In [10]:
heatidx_proj_hour = get_closest_gridcell(heatidx_proj_hour, stn_lat, stn_lon)

Closest gridcell to lat: 36.77999, lon: -119.72016 is at lat: 36.783531188964844, lon: -119.73358154296875


Next, we load the data into memory and convert to local time as we did with the historical data, dropping the final incomplete year.

In [11]:
heatidx_proj_hour = convert_to_local_time(heatidx_proj_hour)
heatidx_proj_hour = heatidx_proj_hour.sel({"time": slice("2040-01-01", "2049-01-01")}) # Update this with your own selections to match the data you retrieved!

Data converted to America/Los_Angeles timezone.


Finally, we load the projected Heat Index data into memory.

In [12]:
heatidx_proj_hour = ck.load(heatidx_proj_hour)

Processing data to read 2.41 MB of data into memory... Complete!


## Step 2: Calculate the number of hours throughout the year above a threshold

### 2a) Sum the hours per day

Let's next determine what part of the day is above a Heat Index threshold, as well as how many hours in each day are above our designated threshold. The NOAA Heat Index  "kicks in" once the [NOAA Heat Index](https://www.noaa.gov/sites/default/files/2022-05/heatindex_chart_rh.pdf) value is above 80°F: prolonged exposure to a Heat Index above 80°F becomes dangerous to many people and especially vulnerable communities. The Occupational Safety and Health Administration (OSHA) uses the Heat Index to determine the risk of heat-related illness and protections for outdoor workers. Below are the [specific thresholds used by OSHA](https://www.nalc.org/workplace-issues/body/OSHA-Using-the-Heat-Index-A-Guide-for-Employers.pdf). We note that these values are slightly different than those used by the [NOAA Heat Index](https://www.noaa.gov/sites/default/files/2022-05/heatindex_chart_rh.pdf) system. We **strongly recommend** looking at multiple thresholds to understand Heat Index trends. 

| Classification | Heat Index |
|----------------|------------|
| Caution | <91°F |
| Moderate | 91 - 103°F |
| High | 103 - 115°F |
| Very High to Extreme | 115+°F |

We'll start with 80°F as our default threshold.

In [13]:
hi_threshold = 80

In [14]:
# counts the number of hours in each day above the heat index threshold
num_heatidx_histhours = (heatidx_hist_hour >= hi_threshold).resample(time="1D").sum()
num_heatidx_histhours.name = (
    f"Hours per day above Heat Index threshold of {hi_threshold}°F"
)

Let's identify one year to visualize the cycle in Heat Index temperatures. We are selecting 1985 here, but the commented out line of code below illustrates how to look at all of the data here. Just comment out the first line under visualize by adding a `#` symbol, and uncommenting the line below by removing the `#` symbol.

In [15]:
data_one_year = num_heatidx_histhours.sel(time="1985")

# visualize
data_one_year.hvplot.line(x="time", by="simulation")  # a specific year
# num_heatidx_histhours.hvplot.line(x='time', by='simulation') # all years

:DynamicMap   [scenario]
   :NdOverlay   [simulation]
      :Curve   [time]   (Hours per day above Heat Index threshold of 80°F)

Heat Index values above our selected threshold (default is 80°F) begin to pick up in March for Fresno, and tail off in November. While high Heat Index values are critical to know during the summer months, we should be aware of high Heat Index values throughout the rest of the year as well. 

### 2b) Sum the total hours per year
It may also be useful to know how many hours in the entire year are above the threshold in order to see the trends over time. We'll calculate this next.

In [16]:
# sum per year
num_heatidx_hist_hours_per_year = num_heatidx_histhours.groupby("time.year").sum("time")
num_heatidx_hist_hours_per_year.name = (
    f"Hours per year above Heat Index threshold of {hi_threshold}°F"
)

In [17]:
# visualize
num_heatidx_hist_hours_per_year.hvplot.line(x="year", by="simulation")

:DynamicMap   [scenario]
   :NdOverlay   [simulation]
      :Curve   [year]   (Hours per year above Heat Index threshold of 80°F)

For context, there are 8760 hours in a year. Let's also look at the future data to understand the projected trends in the number of high Heat Index hours per year. 

In the following cells, we'll do all of the computation above in a single go, to condense the number of cells to run. You don't need to modify anything, unless you made changes above. 

In [18]:
# counts the number of hours in each day above the heat index threshold
num_heatidx_projhours = (heatidx_proj_hour >= hi_threshold).resample(time="1D").sum()
num_heatidx_projhours.name = (
    f"Hours per day above Heat Index threshold of {hi_threshold}°F"
)

# sum per year
num_heatidx_proj_hours_per_year = num_heatidx_projhours.groupby("time.year").sum("time")
num_heatidx_proj_hours_per_year.name = (
    f"Hours per year above Heat Index threshold of {hi_threshold}°F"
)

In [19]:
# visualize
num_heatidx_proj_hours_per_year.hvplot.line(x="year", by="simulation")

:DynamicMap   [scenario]
   :NdOverlay   [simulation]
      :Curve   [year]   (Hours per year above Heat Index threshold of 80°F)

### 2c) Export counts of hours per year
First, we'll calculate summary statistics for our data object with a handy function `compute_multimodel_stats`, which will provide the min, max, mean, and median of our data. We'll also use a helper function `summary_table` which will transform our data objects into an easy-to-read dataframe. This dataframe will be exported as a .csv file and will be available in the file tree on the lefthandside of your screen. You can double click on it to view in a new window, or right-click to download. 

In [20]:
# calculate statistics
num_heatidx_hist_hours_per_year = compute_multimodel_stats(
    num_heatidx_hist_hours_per_year
)

# table format with summary stats
df_to_export = summary_table(num_heatidx_hist_hours_per_year)
df_to_export.head(5)  # see first 5 rows

Hours per year above Heat Index threshold of 80°F  \
simulation                            WRF_EC-Earth3_r1i1p1f1   
scenario                                  Historical Climate   
year                                                           
1981                                                  1581.0   
1982                                                  1497.0   
1983                                                  1414.0   
1984                                                  1637.0   
1985                                                  1967.0   

                                                            \
simulation WRF_FGOALS-g3_r1i1p1f1 WRF_CNRM-ESM2-1_r1i1p1f2   
scenario       Historical Climate       Historical Climate   
year                                                         
1981                       1363.0                   1187.0   
1982                       1027.0                   1150.0   
1983                        853.0                   1260.0   
1984                       1220.0                    993.0   
1985                       1139.0                   1216.0   

                                                                               \
simulation WRF_CESM2_r11i1p1f1 WRF_MIROC6_r1i1p1f1 WRF_MPI-ESM1-2-HR_r3i1p1f1   
scenario    Historical Climate  Historical Climate         Historical Climate   
year                                                                            
1981                    1349.0              1415.0                     1448.0   
1982                    1264.0              1401.0                     1419.0   
1983                    1227.0              1454.0                     1230.0   
1984                    1290.0              1717.0                     1254.0   
1985                    1315.0              1459.0                     1628.0   

                                                                               \
simulation WRF_TaiESM1_r1i1p1f1 WRF_EC-Earth3-Veg_r1i1p1f1    simulation mean   
scenario     Historical Climate         Historical Climate Historical Climate   
year                                                                            
1981                     1697.0                     1283.0           1415.375   
1982                     1477.0                     1688.0           1365.375   
1983                     1322.0                     1518.0           1284.750   
1984                     1180.0                     1473.0           1345.500   
1985                     1589.0                     1651.0           1495.500   

                                                                     
simulation     simulation min     simulation max  simulation median  
scenario   Historical Climate Historical Climate Historical Climate  
year                                                                 
1981                   1187.0             1697.0             1389.0  
1982                   1027.0             1688.0             1410.0  
1983                    853.0             1518.0             1291.0  
1984                    993.0             1717.0             1272.0  
1985                   1139.0             1967.0             1524.0

In [21]:
stn_str = station_name.replace(" ", "_").replace("(", "").replace(")", "")
fn = f"num_heatidx_hours_per_year_{stn_str}"

df_to_export.to_csv(f"{fn}.csv")

### 2d) Calculate the number of Heat Index days per month
Alternatively, let's determine the number of days per month with Heat Index values above the threshold, as this may be useful for seasonal information. From the hourly Heat Index data, we'll now calculate the daily maximum Heat Index value. It is important to note that we will calculate the daily max Heat Index from the hourly Heat Index data, rather than taking the daily max air temperature and the average relative humidity, which artificially inflates the daily Heat Index value. 

If the daily median Heat Index is more relevant to your needs, we also provide the option in the cell below to calculate this instead. 

In [22]:
# historical
heatidx_hist_day = heatidx_hist_hour.resample(time="1D").max()  # daily max
# heatidx_hist_day = heatidx_hist_hour.resample(time='1D').median() # daily median

# future
heatidx_proj_day = heatidx_proj_hour.resample(time="1D").max()  # daily max
# heatidx_proj_day = heatidx_proj_hour.resample(time='1D').median() # daily median

We'll also reset our Heat Index threshold to be 91°F to assess a higher threshold. 

In [23]:
hi_threshold = 91

In [24]:
num_heatidx_histmonths = (heatidx_hist_day >= hi_threshold).resample(time="1ME").sum()
num_heatidx_histmonths.name = (
    f"Days per month above Heat Index threshold of {hi_threshold}°F"
)

In [25]:
num_heatidx_histmonths.hvplot.line(x="time", by="simulation")

:DynamicMap   [scenario]
   :NdOverlay   [simulation]
      :Curve   [time]   (Days per month above Heat Index threshold of 91°F)

### 2e) Export the counts of days per month of high Heat Index values

Like steps 2c, we'll calculate the min, max, mean, and median trends and format a table for easy export. 

In [26]:
# calculate summary stats
num_heatidx_histmonths_stats = compute_multimodel_stats(num_heatidx_histmonths)

# table format with summary stats
df_to_export = summary_table(num_heatidx_histmonths_stats)
df_to_export.head(5)  # see first 5 rows

Days per month above Heat Index threshold of 91°F  \
simulation                            WRF_EC-Earth3_r1i1p1f1   
scenario                                  Historical Climate   
time                                                           
1981-01-31                                               0.0   
1981-02-28                                               0.0   
1981-03-31                                               0.0   
1981-04-30                                               0.0   
1981-05-31                                               0.0   

                                                            \
simulation WRF_FGOALS-g3_r1i1p1f1 WRF_CNRM-ESM2-1_r1i1p1f2   
scenario       Historical Climate       Historical Climate   
time                                                         
1981-01-31                    0.0                      0.0   
1981-02-28                    0.0                      0.0   
1981-03-31                    0.0                      0.0   
1981-04-30                    0.0                      0.0   
1981-05-31                    2.0                      0.0   

                                                                               \
simulation WRF_CESM2_r11i1p1f1 WRF_MIROC6_r1i1p1f1 WRF_MPI-ESM1-2-HR_r3i1p1f1   
scenario    Historical Climate  Historical Climate         Historical Climate   
time                                                                            
1981-01-31                 0.0                 0.0                        0.0   
1981-02-28                 0.0                 0.0                        0.0   
1981-03-31                 0.0                 0.0                        0.0   
1981-04-30                 0.0                 0.0                        0.0   
1981-05-31                 2.0                 0.0                        0.0   

                                                                               \
simulation WRF_TaiESM1_r1i1p1f1 WRF_EC-Earth3-Veg_r1i1p1f1    simulation mean   
scenario     Historical Climate         Historical Climate Historical Climate   
time                                                                            
1981-01-31                  0.0                        0.0              0.000   
1981-02-28                  0.0                        0.0              0.000   
1981-03-31                  0.0                        0.0              0.000   
1981-04-30                  2.0                        0.0              0.250   
1981-05-31                  3.0                        0.0              0.875   

                                                                     
simulation     simulation min     simulation max  simulation median  
scenario   Historical Climate Historical Climate Historical Climate  
time                                                                 
1981-01-31                0.0                0.0                0.0  
1981-02-28                0.0                0.0                0.0  
1981-03-31                0.0                0.0                0.0  
1981-04-30                0.0                2.0                0.0  
1981-05-31                0.0                3.0                0.0

In [27]:
stn_str = station_name.replace(" ", "_").replace("(", "").replace(")", "")
fn = f"num_heatidx_histmonths_{stn_str}"

df_to_export.to_csv(f"{fn}.csv")

## Step 3: Understand trends in nighttime temperatures
Next we'll look specifically at nighttime temperatures in order to assess when it may be too hot outside for worker safety and for assets to cool down. 

### 3a) Subset for nighttime hours
First, let's subset our hourly Heat Index data specifically for the nighttime hours. We will use 8pm-6am as "nighttime", but you can modify based on your needs as well.

In [28]:
night_subset = [20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6]  # 24 hour time
heatidx_hist_nighthours = heatidx_hist_hour.isel(
    time=heatidx_hist_hour.time.dt.hour.isin(night_subset)
)

### 3b) Sum the number of nighttime hours above a threshold of 80°F per day and per year
Like what we did above in Step 2a and 2b, we'll sum the nighttime Heat Index values for analysis. We'll use the same threshold as above, 91°F. But you can modify easily by setting this to any value of interest.

In [29]:
# sum per day
num_heatidx_hist_nighthours = (
    (heatidx_hist_nighthours >= hi_threshold).resample(time="1D").sum()
)
num_heatidx_hist_nighthours.name = (
    f"Nighttime hours per day above Heat Index threshold of {hi_threshold}°F"
)

In [30]:
# visualize
num_heatidx_hist_nighthours.hvplot.line(x="time", by="simulation")

:DynamicMap   [scenario]
   :NdOverlay   [simulation]
      :Curve   [time]   (Nighttime hours per day above Heat Index threshold of 91°F)

In [31]:
# sum per year
num_heatidx_hist_nighthours_per_year = num_heatidx_hist_nighthours.groupby(
    "time.year"
).sum("time")
num_heatidx_hist_nighthours_per_year.name = (
    f"Nighttime hours per year above Heat Index threshold of {hi_threshold}°F"
)

In [32]:
# visualize
num_heatidx_hist_nighthours_per_year.hvplot.line(x="year", by="simulation")

:DynamicMap   [scenario]
   :NdOverlay   [simulation]
      :Curve   [year]   (Nighttime hours per year above Heat Index threshold of 91°F)

We'll calculate the same steps for the projected future data, but won't visualize it here. We're just including it in case you need this information.

In [33]:
# subset for nighttime hours
heatidx_proj_nighthours = heatidx_proj_hour.isel(
    time=heatidx_proj_hour.time.dt.hour.isin(night_subset)
)

# sum per day
num_heatidx_proj_nighthours = (
    (heatidx_proj_nighthours >= hi_threshold).resample(time="1D").sum()
)
num_heatidx_proj_nighthours.name = (
    f"Nighttime hours per day above Heat Index threshold of {hi_threshold}°F"
)

# sum per year
num_heatidx_proj_nighthours_per_year = num_heatidx_proj_nighthours.groupby(
    "time.year"
).sum("time")
num_heatidx_proj_nighthours_per_year.name = (
    f"Nighttime hours per year above Heat Index threshold of {hi_threshold}°F"
)

### 3c) Export counts of nighttime hours per year

Similar the above export steps, we'll export the counts of nighttime temperatures per year next. 

In [34]:
# calculate summary stats
num_heatidx_hist_nighthours_per_year = compute_multimodel_stats(
    num_heatidx_hist_nighthours_per_year
)

# table format with summary stats
df_to_export = summary_table(num_heatidx_hist_nighthours_per_year)
df_to_export.head(5)  # see first 5 rows

Nighttime hours per year above Heat Index threshold of 91°F  \
simulation                                      WRF_EC-Earth3_r1i1p1f1   
scenario                                            Historical Climate   
year                                                                     
1981                                                      5.0            
1982                                                      5.0            
1983                                                      3.0            
1984                                                      1.0            
1985                                                     15.0            

                                                            \
simulation WRF_FGOALS-g3_r1i1p1f1 WRF_CNRM-ESM2-1_r1i1p1f2   
scenario       Historical Climate       Historical Climate   
year                                                         
1981                          0.0                      0.0   
1982                          0.0                      1.0   
1983                          0.0                      0.0   
1984                          0.0                      0.0   
1985                          0.0                      0.0   

                                                                               \
simulation WRF_CESM2_r11i1p1f1 WRF_MIROC6_r1i1p1f1 WRF_MPI-ESM1-2-HR_r3i1p1f1   
scenario    Historical Climate  Historical Climate         Historical Climate   
year                                                                            
1981                       1.0                 3.0                       12.0   
1982                       0.0                 0.0                        5.0   
1983                       0.0                 0.0                        0.0   
1984                       4.0                 6.0                        4.0   
1985                       1.0                 4.0                        6.0   

                                                                               \
simulation WRF_TaiESM1_r1i1p1f1 WRF_EC-Earth3-Veg_r1i1p1f1    simulation mean   
scenario     Historical Climate         Historical Climate Historical Climate   
year                                                                            
1981                       23.0                        0.0              5.500   
1982                        1.0                       15.0              3.375   
1983                        4.0                        5.0              1.500   
1984                        2.0                        0.0              2.125   
1985                        3.0                        9.0              4.750   

                                                                     
simulation     simulation min     simulation max  simulation median  
scenario   Historical Climate Historical Climate Historical Climate  
year                                                                 
1981                      0.0               23.0                2.0  
1982                      0.0               15.0                1.0  
1983                      0.0                5.0                0.0  
1984                      0.0                6.0                1.5  
1985                      0.0               15.0                3.5

In [35]:
stn_str = station_name.replace(" ", "_").replace("(", "").replace(")", "")
fn = f"num_heatidx_nighthours_per_year_{stn_str}"
df_to_export.to_csv(f"{fn}.csv")

## Step 4: Summarize the long term trends

### 4a) Comparing historical and projected number of days in each year above a Heat Index threshold

Using the [OSHA thresholds we noted above](https://www.nalc.org/workplace-issues/body/OSHA-Using-the-Heat-Index-A-Guide-for-Employers.pdf), we'll now look at a threshold of 91°F. Again, we **strongly recommend** looking at multiple thresholds to understand Heat Index trends.

In [36]:
hi_threshold = 91  # degF

If you would like to look at a specific month or season, uncomment the next cell below. We are going to look at the entire year, and leave this cell commented out. You can also skip the next cell entirely if you want to retain information throughout the entire year. 

In [37]:
# month_subset = [5, 6, 7, 8, 9] # May, June, July, August, September
# heatidx_hist_day = heatidx_hist_day.isel(time = heatidx_hist_day.time.dt.month.isin(month_subset)) # historical
# heatidx_proj_day = heatidx_proj_day.isel(time = heatidx_proj_day.time.dt.month.isin(month_subset)) # future

Let's visualize the historical trends next. 

In [38]:
ckg.view(heatidx_hist_day)

:DynamicMap   [scenario,simulation]
   :Curve   [time]   (NOAA Heat Index)

In [39]:
ckg.view(heatidx_proj_day)

:DynamicMap   [scenario,simulation]
   :Curve   [time]   (NOAA Heat Index)

Compare the median historical and projected Heat Index values.

In [40]:
# what is the projected change in the median daily max heat index value?
hi_hist = heatidx_hist_day.median().values
hi_proj = heatidx_proj_day.median().values
print(f"Historical median Heat Index: {hi_hist:.2f}")
print(f"Projected median Heat Index: {hi_proj:.2f}")
print(
    f"The projected change in the median Heat Index value from historical is: {(hi_proj - hi_hist):.2f}°F"
)

Historical median Heat Index: 0.00
Projected median Heat Index: 0.00
The projected change in the median Heat Index value from historical is: -0.00°F


### 4b) Compare the historical and projected number of days above the threshold

In [41]:
# calculate number of days above threshold
num_heatidx_histdays = (
    (heatidx_hist_day >= hi_threshold).groupby("time.year").sum("time")
)
num_heatidx_histdays.name = f"Days above Heat Index threshold of {hi_threshold}°F"

# calculate summary statistics
num_heatidx_histdays_stats = compute_multimodel_stats(num_heatidx_histdays)

Let's visualize the trends:

In [42]:
num_heatidx_histdays.hvplot.line(x="year", by="simulation", title="") * trendline(
    num_heatidx_histdays_stats, kind="median"
).hvplot.line(x="year", color="black", line_dash="dashed", label="trendline")

:DynamicMap   [scenario]
   :Overlay
      .NdOverlay.I     :NdOverlay   [simulation]
         :Curve   [year]   (Days above Heat Index threshold of 91°F)
      .Curve.Trendline :Curve   [year]   (trendline)

Note, if you've modified the threshold to be a higher value, for example 105°F, if the per-model trends and the median trend return flat lines, this indicates that for this location, the Heat Index value has not exceeded the modified threshold in the modeled data.

We'll now repeat the process for the projected data.

In [43]:
# calculate number of days above the threshold
num_heatidx_projdays = (
    (heatidx_proj_day >= hi_threshold).groupby("time.year").sum("time")
)
num_heatidx_projdays.name = f"Days above Heat Index threshold of {hi_threshold}°F"

# calculate statistics
num_heatidx_projdays_stats = compute_multimodel_stats(num_heatidx_projdays)

In [44]:
# visualize results
num_heatidx_projdays.hvplot.line(x="year", by="simulation", title="") * trendline(
    num_heatidx_projdays_stats, kind="median"
).hvplot.line(x="year", color="black", line_dash="dashed", label="trendline")

:DynamicMap   [scenario]
   :Overlay
      .NdOverlay.I     :NdOverlay   [simulation]
         :Curve   [year]   (Days above Heat Index threshold of 91°F)
      .Curve.Trendline :Curve   [year]   (trendline)

### 4c) Export the number of days above a threshold

In [45]:
# table format with summary stats
df_to_export = summary_table(num_heatidx_projdays_stats)
df_to_export.head(5)  # see first 5 rows

Days above Heat Index threshold of 91°F                             \
simulation                  WRF_FGOALS-g3_r1i1p1f1 WRF_EC-Earth3-Veg_r1i1p1f1   
scenario                                 SSP 3-7.0                  SSP 3-7.0   
year                                                                            
2040                                          25.0                       58.0   
2041                                          17.0                       83.0   
2042                                          21.0                       94.0   
2043                                          48.0                       55.0   
2044                                          47.0                       69.0   

                                                                              \
simulation WRF_CNRM-ESM2-1_r1i1p1f2 WRF_TaiESM1_r1i1p1f1 WRF_CESM2_r11i1p1f1   
scenario                  SSP 3-7.0            SSP 3-7.0           SSP 3-7.0   
year                                                                           
2040                           50.0                 92.0                56.0   
2041                           58.0                 87.0                55.0   
2042                           36.0                118.0                69.0   
2043                           57.0                 77.0                71.0   
2044                           52.0                107.0                40.0   

                                                       \
simulation WRF_EC-Earth3_r1i1p1f1 WRF_MIROC6_r1i1p1f1   
scenario                SSP 3-7.0           SSP 3-7.0   
year                                                    
2040                         97.0                67.0   
2041                         79.0                70.0   
2042                         76.0                75.0   
2043                         75.0                62.0   
2044                        101.0                79.0   

                                                                      \
simulation WRF_MPI-ESM1-2-HR_r3i1p1f1 simulation mean simulation min   
scenario                    SSP 3-7.0       SSP 3-7.0      SSP 3-7.0   
year                                                                   
2040                             79.0          65.500           25.0   
2041                             76.0          65.625           17.0   
2042                             84.0          71.625           21.0   
2043                             71.0          64.500           48.0   
2044                             53.0          68.500           40.0   

                                             
simulation simulation max simulation median  
scenario        SSP 3-7.0         SSP 3-7.0  
year                                         
2040                 97.0              62.5  
2041                 87.0              73.0  
2042                118.0              75.5  
2043                 77.0              66.5  
2044                107.0              61.0

In [46]:
stn_str = station_name.replace(" ", "_").replace("(", "").replace(")", "")
fn = f"num_heatidx_projdays_stats_{stn_str}"
df_to_export.to_csv(f"{fn}.csv")

**Note**: Any of the data variables that we've calculated throughout this notebook can be exported to .csv, like we've demonstrated throughout this notebook. Feel free to use this code to export any other variables of interest.

## Step 5: Export WRF data
In the cell below we also provide the optionality to export the input WRF data too. We demonstrate with the historical hourly Heat Index values here, but you can replace `heatidx_hist_hour` with your dataset of choice, including `heatidx_proj_hour`!

In [47]:
ck.export(heatidx_hist_hour, "my_filename", "netcdf")

Exporting specified data to NetCDF...
Saving file locally as NetCDF4...
[########################################] | 100% Completed | 102.71 ms
Saved! You can find your file in the panel to the left and download to your local machine from there.
